## Importing Libraries

In [ ]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

## API & Connections

In [ ]:
api_key = 'Your API Key'

channel_ids = [ 
                'UCLLw7jmFsvfIVaUFsLs8mlQ',   # Luke Barousse
                'UCiT9RITQ9PW6BhXK0y2jaeg',   # Ken Jee
                'UC2UXDak6o7rBm23k3Vv5dww',    # Tina Huang
                'UCnz-ZXXER4jOvuED5trXfEA'    #techTFQ
            ]
youtube = build('youtube', 'v3', developerKey=api_key)

## Function to get channel stats

In [ ]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    for i in range(len(response['items'])):
        data = dict(channel_name= response['items'][i]['snippet']['title'],
                    views= response['items'][i]['statistics']['viewCount'],
                    subscribers= response['items'][i]['statistics']['subscriberCount'],
                    total_videos= response['items'][i]['statistics']['videoCount'],
                    playlist_id= response['items'][i]['contentDetails']['relatedPlaylists']['uploads']

        )
        all_data.append(data)

    return all_data

In [ ]:
channel_stats= get_channel_stats(youtube, channel_ids)

### Creating Channel dataFrame

In [ ]:
channel_data= pd.DataFrame(channel_stats)

In [ ]:
channel_data

## Checking Data Types

In [ ]:
channel_data.dtypes

## Converting Data Types

In [ ]:
channel_data['subscribers'] = pd.to_numeric(channel_data['subscribers'])
channel_data['views'] = pd.to_numeric(channel_data['views'])
channel_data['total_videos'] = pd.to_numeric(channel_data['total_videos'])

In [ ]:
channel_data.dtypes

## Subscribers Count

In [ ]:
sns.set(rc={'figure.figsize':(10,4)})
subs = sns.barplot(x='channel_name', y='subscribers', data= channel_data, palette="pastel")

## Views Count

In [ ]:
sns.set(rc={'figure.figsize':(10,4)})
views = sns.barplot(x='channel_name', y='views', data= channel_data, palette="pastel")

## Videos Count

In [ ]:
sns.set(rc={'figure.figsize':(10,4)})
num_of_videos = sns.barplot(x='channel_name', y='total_videos', data= channel_data, palette="pastel")

## Scrape, Analyze, Visualize (Ken Jee Channel)

#### Fetch Video Id

In [ ]:
playlist_id= channel_data.loc[channel_data['channel_name']=='Ken Jee', 'playlist_id'].iloc[0]
playlist_id

### Creating Function

In [ ]:
def get_video_ids(youtube, playlist_id):

    request = youtube.playlistItems().list(
        part= 'contentDetails',
        playlistId= playlist_id, maxResults= 50)

    response = request.execute()
    video_ids = []
     
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token= response.get('nextPageToken')
    more_pages = True

    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
            part= 'contentDetails',
            playlistId= playlist_id, maxResults= 50, pageToken= next_page_token
            )
            response = request.execute() 

            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
            next_page_token =response.get('nextPageToken')

    return video_ids

### Getting Videos ID

In [ ]:
video_ids= get_video_ids(youtube, playlist_id)

In [ ]:
video_ids

## Function to get video details

In [ ]:
def get_video_details(youtube, video_ids):
    all_video_stats =[]
    for i in range(0,len(video_ids),50):
        request = youtube.videos().list(
         part = 'snippet, statistics',
            id = ','.join(video_ids[i:i+50]))
        response = request.execute()

        for video in response['items']:
            video_stats = dict(Title= video['snippet']['title'],
                               published_date = video['snippet'] ['publishedAt'],
                               views = video['statistics'] ['viewCount'],
                               likes = video['statistics'] ['likeCount'],
                            #    dislikes = video['statistics'] ['dislikeCount'], #This API has been revoked by YT since dec 2021
                               comments = video['statistics'] ['commentCount'])
            all_video_stats.append(video_stats)
    return all_video_stats

In [ ]:
video_details= get_video_details(youtube, video_ids)

In [ ]:
video_data = pd.DataFrame(video_details)

### Converting Data Types

In [ ]:
video_data['published_date']= pd.to_datetime(video_data['published_date']).dt.date
video_data['views']= pd.to_numeric(video_data['views'])
video_data['likes']= pd.to_numeric(video_data['likes'])
video_data['comments']= pd.to_numeric(video_data['comments'])

### Top 5 Videos

In [ ]:
top_5_videos = video_data.sort_values(by='views', ascending=False).head()

In [ ]:
top_5_videos

### Top 5 Videos- BarPlot

In [ ]:
top_5_videos_barchart = sns.barplot(x='views', y= 'Title', data=top_5_videos, palette="pastel")

### Converting Data Types

In [ ]:
video_data['month']= pd.to_datetime(video_data['published_date']).dt.strftime('%b')

In [ ]:
video_data

### Grouping Videos by Months

In [ ]:
videos_per_month= video_data.groupby('month', as_index=False).size()

In [ ]:
videos_per_month

### Sorting Month Order

In [ ]:
sort_order= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


In [ ]:
videos_per_month.index = pd.CategoricalIndex(videos_per_month['month'], categories= sort_order, ordered=True)

In [ ]:
videos_per_month= videos_per_month.sort_index()

### Videos by Month - BarPlot

In [ ]:

videos_per_month_barplot= sns.barplot(x='month', y='size', data=videos_per_month, palette="pastel")

## Creating CSV file

In [ ]:
video_data.to_csv('Video Details(Ken Jee).csv')